In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import torch
from torch import nn
import random
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import json
from collections import namedtuple
from sklearn.metrics import classification_report
import cv2

In [6]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  resize = 224

  transform_train = transforms.Compose([
      
      transforms.Resize((resize, resize)),
      #Preprocess: Can bang histogram
      #transforms.RandomEqualize(p=0.5),


      # Augmentation:Xoay anh, bop meo anh
      #transforms.RandAugment(num_ops=3),
      #transforms.RandomAffine(degrees=90, translate=None, shear=25),

      transforms.ToTensor()
      #transforms.Normalize(mean, std)
  ])

  transform_test = transforms.Compose([
    
    transforms.Resize((resize, resize)),
    
    #transforms.RandomEqualize(p=0.2),
    #transforms.RandAugment(num_ops=3),
    # transforms.RandomAffine(degrees=45, translate=None, shear=15),
    
    transforms.ToTensor(),
    #transforms.Normalize(mean, std)
  ])

  trainset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/test', transform=transform_test)

  print("Number of Image in train set:", len(trainset))
  print("Number of Image in test set:", len(testset))
  print("Classes: ", trainset.class_to_idx)

  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  print("Num batch in train set: ", len(trainloader))
  return TrainTest(train=trainloader, test=testloader)

def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 20
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if step % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue



In [7]:
def main(MODEL='vgg16', type='img'):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0")
  use_pretrained = False

  if MODEL == 'vgg19':
    PATH='./vgg19.pth'
    model = models.vgg19(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3)
  elif MODEL == 'resnet50':
    PATH='./resnet50.pth'
    model = models.resnet50(pretrained=use_pretrained)
    model.fc = nn.Linear(in_features=2048, out_features=3)
  else:
    PATH='./vgg16.pth'
    model = models.vgg16(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3) 

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)


  if type=='img':

    print(classification_report(ytrue, ypred, target_names=classes))
    torch.save(model.state_dict(), PATH)
    return model

  elif type=='video':
    ytrue = np.array([10])
    ypred = np.array([10])
    vid = -1
    true = 0
    count = np.array([0,0,0])
    predict = 0
    
    for i in range (len(datasets.test.imgs)):
      if (datasets.test.imgs[i][0].split('C/')[1].split('_')[0] != vid):
        if ((count[0] != 0) | (count[1] != 0) | (count[2] != 0)):
          max_ = count.max()
          for j in range (len(count)):
            if (count[j] == max_):
              predict = j
              break 
          print('video:',vid, '  true:', true, '  predic:',predict)
          ytrue_ = np.append(ytrue_, true)
          ypred_ = np.append(ypred_, predict)
          count[:] = 0
        video = datasets.test.imgs[i][0].split('C/')[1].split('_')[0]
        true = ytrue[i] 
      count[ypred[i]] += 1
    print('video:',vid, '  true:', true, '  predic:',predic)
    ytrue_ = np.append(ytrue_, true)
    ypred_ = np.append(ypred_, predict)
    ytrue_ = np.delete(ytrue_, 0)
    ypred_ = np.delete(ypred_, 0)

    print(classification_report(ytrue_, ypred_, target_names=classes))



In [ ]:
model_VGG16 = main(MODEL='vgg16')

In [ ]:
model_VGG19 = main(MODEL='vgg19')

In [ ]:
model_Resnet50 = main(MODEL='resnet50')

In [ ]:
video = main(MODEL='vgg16', type='video')